In [ ]:
# This jupyter note book helps play around with queries manually

In [32]:
from dotenv import load_dotenv
import os
from csdash import dbaccess
from rich import print

In [33]:
# load the dependencies and check instaleld correctly 
load_dotenv()

es = dbaccess.connect_cogstack()
user = os.environ.get("ELASTIC_API_ID")
print(user)

TbF70ZQBjIvyPy2Q2836

In [34]:

index = "gstt_clinical_epic_notes_20250123"

query3 = {
  "size":0,
  "aggs":{
    "unique_values_activity_specialtyadmit":{
      "terms":{
        "field":"activity_AdmittingProviderSpecialty",
        "size": 10000
      }
    },
    "unique_values_doc_deptspecialty":{
      "terms":{
        "field":"activity_DepartmentSpecialty",
        "size": 10000
      }
    },
    "unique_vals_ptclass":{
      "terms":{
        "field":"activity_PatientClass",
        "size":10000
      }
    },
    "unique_activity_type":{
      "terms":{
        "field":"activity_Type",
        "size":10000
      }
    },
    "doc_authour_vals":{
      "terms":{
        "field":"document_AuthorType",
        "size":10000
      }
    },
    "careteam":{
      "terms":{
        "field":"activity_CareTeam",
        "size":10000
      }
    }
  }
}


In [ ]:
response = es.search(index=index, body=query3)

In [ ]:
print(response["aggregations"])

In [42]:
query4 = {
  "_source": [
    "activity_AdmissionDate",
    "activity_Date",
    "activity_CreatedWhen",
    "activity_DischargeDate",
    "activity_VisitType",
    "activity_VisitClass"
    "activity_AdmittingProvider",
    "activity_AdmittingProviderSpecialty",
    "activity_CareTeam",
    "activity_DepartmentSpecialty",
    "activity_Type",
    "activity_PatientClass",
    "document_Author",
    "document_AuthorSpecialty",
    "document_AuthorType",
    "id",
    "document_CreatedWhen"
    "document_Name",
    "document_Content",
    "document_CreatedWhen",
    "document_UpdatedWhen",
    "patient_HospitalNumber",
    "patient_MRN",
    "patient_SourceId",
    "patient_NhsNumber",
    "patient_FirstName",
    "patient_MiddleName",
    "patient_LastName",
    "patient_Gender",
    "patient_DateOfBirth",
    "patient_Ethnicity",
    "patient_EpicId"
  ],
  "query": {
    "bool": {
      "must":[],
      "filter":[
        {
            "terms":{
                "activity_DepartmentSpecialty":[
                    "Dermatology",
                    "Medical Oncology",
                    "Clinical Oncology",
                    "Gynaecology",
                    "Clinical Haematology",
                    "Oral Surgery",
                    "Pre-Assessment",
                    "Endoscopy",
                    "Gynaecological Oncology",
                    "Breast Surgery",
                    "Clinical Genetics",
                    "ENT - Head and Neck",
                    "Palliative Care",
                    "Cancer Supportive Services",
                    "Gynaecology - Urogynaecology",
                    "Chemical Pathology"
                ]
            }
        },
        {
          "terms":{
            "activity_PatientClass":[
              "Outpatient",
              "Inpatient",
              "Emergency",
              "Day Case",
              "Same Day Emergency Care",
              "Surgery Admit"
            ]
          }
        },
        {
          "terms":{
            "activity_Type":[
              "Appointment",
              "Hospital Encounter",
              "Clinic/Practice Visit",
              "Documentation",
              "MDT Meeting",
              "Referral Triage",
              "Group Documentation",
              "Enrolment",
              "Letter (Out)",
              "Results Follow-Up",
              "Scanned Document",
              "Consult",
              "Follow-up",
              "Treatment",
              "Operation",
              "Clinical Documentation Only"
            ]
          }
        },
        {
          "terms":{
            "activity_CareTeam":[
              "STH PALLIATIVE CARE",
              "GH PT ONCOLOGY TEAM",
              "GH OT ONCOLOGY TEAM",
              "GH PALLIATIVE CARE",
              "GH MEDICAL ONCOLOGY TEAM A",
              "STH END OF LIFE",
              "GH MEDICAL ONCOLOGY TEAM B",
              "GH HAEMATOLOGY (ONCOLOGY)",
              "GH MEDICAL ONCOLOGY TEAM C",
              "GH END OF LIFE",
              "RBH PALLIATIVE CARE",
              "STH ACUTE ONCOLOGY",
              "STH GYNAECOLOGY ONCOLOGY",
              "GH SLT ONCOLOGY TEAM",
              "STH DERMATOLOGY",
              "STH GYNAECOLOGY - ELECTIVE",
              "GH SLT ONCOLOGY TEAM",
              "GH HAEMATOLOGY/ONCOLOGY (HOSPITAL-AT-NIGHT/OUT-OF-HOURS)",
              "GH / STH MEDICAL ONCOLOGY - UROLOGY AND GYNAECOLOGY",
              "GH / STH CLINICAL ONCOLOGY - HEAD AND NECK",
              "STH / GH HAEMATOLOGY GENERAL",
              "GH ACUTE ONCOLOGY",
              "DH HAEMATOLOGY",
              "DH ACUTE ONCOLOGY",
              "DH MEDICAL ONCOLOGY",
              "DH CLINICAL ONCOLOGY",
              "GH / STH CLINICAL ONCOLOGY - GYNAE",
              "DH GYNAECOLOGY",
              "DH HAEMATOLOGY – MYELOID TEAM A",
              "DH GYNAECOLOGICAL ONCOLOGY",
              "GH / STH MEDICAL ONCOLOGY - LUNG",
              "GH / STH CLINICAL ONCOLOGY - LUNG",
            ]
          }
        },
        {
          "terms":{
            "document_AuthorType":[
              "Registered Nurse",
              "Consultant",
              "Clinical Nurse Specialist",
              "Clinical Fellow",
              "Specialty Trainee",
              "Core Trainee",
              "Foundation Doctor (Year 1)",
              "Doctor",
              "Advanced Nurse Practitioner",
              "Specialty Doctor",
              "Pharmacist",
              "Foundation Doctor (Year 2)",
              "Physician Associate",
              "Dental Core Trainee",
              "Emergency Nurse Practitioner",
              "Specialty Dentist",
              "Anaesthesia Specialty Trainee",
              "Dentist",
              "Consultant Dentist",
              "Anaesthesia Clinical Fellow",
              "Postgraduate Dentist",
              "Research Coordinator",
              "Nurse Consultant",
              "Anaesthesia Core Trainee",
              "Clinical Scientist",
              "Anaesthesia Specialty Doctor",
              "Surgical Care Practitioner",
              "Specialty Doctor (Dental)",
              "Anaesthesia Post-CCT",
              "Biomedical Scientist",
              "Advanced Practitioner",
              "Anaesthesia Associate",
              "Trainee Clinical Scientist",
              "Medical Student"
            ]
          }
        }
      ]
    }
   }
}

In [39]:
response2 = es.search(index=index, body=query4,scroll="2m")
print(response2["hits"]["total"]["value"])

186306

In [40]:
# use custom function to pull queries and convert to dataframes
df = dbaccess.fetch_query(es=es,index_name=index,query=query4)

In [41]:
print(df)

_index           _id  _score  \
0  gstt_clinical_epic_notes_20250123  ecn_21991224     0.0   
1  gstt_clinical_epic_notes_20250123   ecn_6373345     0.0   
2  gstt_clinical_epic_notes_20250123   ecn_6275591     0.0   
3  gstt_clinical_epic_notes_20250123   ecn_6402253     0.0   
4  gstt_clinical_epic_notes_20250123   ecn_6092577     0.0   
5  gstt_clinical_epic_notes_20250123   ecn_5952748     0.0   
6  gstt_clinical_epic_notes_20250123   ecn_5974937     0.0   
7  gstt_clinical_epic_notes_20250123   ecn_6169888     0.0   
8  gstt_clinical_epic_notes_20250123   ecn_5886005     0.0   
9  gstt_clinical_epic_notes_20250123   ecn_6569099     0.0   

  _source.document_AuthorType  \
0           Specialty Trainee   
1            Registered Nurse   
2            Registered Nurse   
3                  Consultant   
4            Registered Nurse   
5                  Pharmacist   
6                  Consultant   
7   Clinical Nurse Specialist   
8            Registered Nurse   
9           Specialty Trainee   

                            _source.document_Content  \
0  Remote review\r\n\r\nHad desaturation today mo...   
1  Care taken over of patient at 8pm from day shi...   
2  Care taken over of patient at 8pm from day shi...   
3  Daily Ward Round\r\n\r\nDate: 21/12/23 Time: 1...   
4  Introduced myself to Ibolya when care taken ov...   
5  Pharmacist Entry re: discharge planning\r\n\r\...   
6  Pharmacy noted actioned - Dalteparin not held ...   
7  Consult\r\nLA Midline inserted. Line is ready ...   
8  NIGHT SHIFT\r\n\r\nDx: Sickle Cell\r\n\r\nS - ...   
9  Haematology WR - SpR AMIN\r\n\r\nJulie is much...   

   _source.patient_SourceId _source.patient_EpicId  _source.patient_NhsNumber  \
0                   2495328               Z5984032                 6241303295   
1                   1905597               Z5513336                 7063681842   
2                   1905597               Z5513336                 7063681842   
3                   1905597               Z5513336                 7063681842   
4                   1905597               Z5513336                 7063681842   
5                   1905597               Z5513336                 7063681842   
6                    205489               Z5737011                 4421211672   
7                    205489               Z5737011                 4421211672   
8                    205489               Z5737011                 4421211672   
9                    205489               Z5737011                 4421211672   

  _source.patient_MRN                     _source.patient_HospitalNumber  ...  \
0            35970686                                          [E001902]  ...   
1            35499990                                [K430072, 8304421B]  ...   
2            35499990                                [K430072, 8304421B]  ...   
3            35499990                                [K430072, 8304421B]  ...   
4            35499990                                [K430072, 8304421B]  ...   
5            35499990                                [K430072, 8304421B]  ...   
6            35723665  [A575864, 2285349Y, 615606, F507453, E001787, ...  ...   
7            35723665  [A575864, 2285349Y, 615606, F507453, E001787, ...  ...   
8            35723665  [A575864, 2285349Y, 615606, F507453, E001787, ...  ...   
9            35723665  [A575864, 2285349Y, 615606, F507453, E001787, ...  ...   

  _source.activity_VisitType  \
0             AOS ASSESSMENT   
1             AOS ASSESSMENT   
2             AOS ASSESSMENT   
3             AOS ASSESSMENT   
4             AOS ASSESSMENT   
5             AOS ASSESSMENT   
6                        NaN   
7                        NaN   
8                        NaN   
9                        NaN   

                           _source.activity_CareTeam  \
0   [GH MEDICAL ONCOLOGY TEAM A, GH PALLIATIVE CARE]   
1  [GH PT ONCOLOGY TEAM, GH MEDICAL ONCOLOGY TEAM...   
2  [GH PT ONCOLOGY TEAM, GH MEDICAL ONCOLOGY TEAM...   
3  [G